# Tiền xử lí dữ liệu sơ bộ

## Chotot
- Dữu liệu thu thập: title,price,Số Km đã đi,Còn hạn đăng kiểm,Xuất xứ,Tình trạng,Chính sách bảo hành,Hãng,Dòng xe,Năm sản xuất,Hộp số,Nhiên liệu,Trọng lượng,Trọng tải,location,seller,Kiểu dáng,Số chỗ,Số đời chủ,Có phụ kiện đi kèm
- Dữ liệu có thể sử dụng: price,Số Km đã đi,Xuất xứ,Tình trạng,Hãng,Dòng xe,Năm sản xuất,Nhiên liệu,Kiểu dáng,Số chỗ,Nhiên liệu,location

## Bonbanh
- Dữu liệu thu thập: title,price,location,Tình trạng,Năm sản xuất,Xuất xứ,Động cơ,Kiểu dáng,Hộp số,Số Km đã đi,Dẫn động,Màu ngoại thất,Màu nội thất,Số cửa,Số chỗ ngồi
- Dữ liệu có thể sử dụng:  title (series nằm bên trong title), price, Năm sản xuất,Tình trạng,Số Km đã đi,Xuất xứ,Kiểu dáng,Động cơ,Số chỗ ngồi, location

## Kết quả cuối
- Quy tắc trường dữ liệu: price, odometer, year, brand, series, status, origin, fuel_type, style, seats, location, engine, gear

# 0. Nguồn dữ liệu

In [ ]:
import pandas as pd
import re
import numpy as np
import os

chotot_df = pd.read_csv('../datasets/raw_chotot_car_features.csv')
bonbanh_df = pd.read_csv('../datasets/raw_bonbanh_car_features.csv')

# ---- Chuẩn hóa cột chung ----
normalized_columns = [
    'price', 'odometer', 'origin', 'status',
    'brand', 'series', 'year', 'fuel_type',
    'style', 'seats', 'location', 'gear', 'engine'
]

print("Chotot shape:", chotot_df.shape)
print("Bonbanh shape:", bonbanh_df.shape)

# 1. Chuẩn hóa tên cột

### 1.1 Chotot

In [ ]:
# ---- Chotot ----
chotot_selected = chotot_df[[  # sửa theo tên cột gốc của bạn
    'price', 'Số Km đã đi', 'Xuất xứ', 'Tình trạng',
    'Hãng', 'Dòng xe', 'Năm sản xuất', 'Nhiên liệu',
    'Kiểu dáng', 'Số chỗ', 'location', "Hộp số"
]].copy()

# Chotot không có engine
chotot_selected['engine'] = np.nan

chotot_selected.columns = normalized_columns


print("Chotot processed with engine (NaN) and series:")
print(chotot_selected.head())

### 1.2. Bobanh

In [ ]:
# ---- Bonbanh ----
bonbanh_selected = bonbanh_df[[  # sửa theo tên cột gốc của bạn
    'title', 'price', 'Năm sản xuất', 'Tình trạng',
    'Số Km đã đi', 'Xuất xứ', 'Kiểu dáng',
    'Động cơ', 'Số chỗ ngồi', 'location'
]].copy()


# Đổi tên các cột còn lại
bonbanh_selected = bonbanh_selected.rename(columns={
    'Năm sản xuất': 'year',
    'Tình trạng': 'status',
    'Số Km đã đi': 'odometer',
    'Xuất xứ': 'origin',
    'Động cơ': 'fuel_type',
    'Kiểu dáng': 'style',
    'Số chỗ ngồi': 'seats'
})

# Tạo các feature trống
bonbanh_selected['gear'] = np.nan
bonbanh_selected['engine'] = np.nan
bonbanh_selected['series'] = np.nan

# ---- Kiểm tra kết quả ----
print("Bonbanh processed with engine and series:")
print(bonbanh_selected.head())

# 2. Chuẩn hóa giá bán

In [ ]:
def parse_vn_price(price_str):
    """
    Chuyển giá tiền từ định dạng tiếng Việt về số nguyên (đồng)
    Ví dụ:
        "2 Tỷ 185 Triệu" -> 2185000000
        "1 Tỷ" -> 1000000000
        "500 Triệu" -> 500000000
    """
    if pd.isna(price_str) or price_str.strip() == '':
        return None

    price_str = price_str.replace('.', '').replace(',', '').strip()

    total = 0

    # Tìm số Tỷ
    ty_match = re.search(r'(\d+)\s*Tỷ', price_str, flags=re.IGNORECASE)
    if ty_match:
        total += int(ty_match.group(1)) * 1_000_000_000  # 1 Tỷ = 1.000.000.000 đồng

    # Tìm số Triệu
    trieu_match = re.search(r'(\d+)\s*Triệu', price_str, flags=re.IGNORECASE)
    if trieu_match:
        total += int(trieu_match.group(1)) * 1_000_000  # 1 Triệu = 1.000.000 đồng

    # Nếu không tìm thấy Tỷ hay Triệu, thử lấy số thẳng
    if total == 0:
        digits = re.findall(r'\d+', price_str)
        if digits:
            total = int(''.join(digits))
        else:
            total = None

    return total

chotot_selected['price'] = (
    chotot_selected['price']
    .astype(str)
    .str.replace(r'\D', '', regex=True)
    .replace('', pd.NA)
    .astype('Int64')
)

bonbanh_selected['price'] = bonbanh_selected["price"].apply(parse_vn_price)


# 3. Chuẩn hóa số Kilometer đã đi

In [ ]:
# Chotot
chotot_selected['odometer'] = (
    pd.to_numeric(chotot_selected['odometer'], errors='coerce')
    .astype('Int64')
)

# Bobanh
bonbanh_selected['odometer'] = (
    bonbanh_selected['odometer']
    .astype(str)
    .str.replace(r'[^0-9]', '', regex=True)
    .pipe(pd.to_numeric, errors='coerce')
    .astype('Int64')
)

# 4. Lấy engine của bonbanh

In [ ]:
# ---- Hàm trích xuất engine từ cột 'Động cơ' Bonbanh ----
def extract_engine(engine_text):
    """
    Trích xuất engine_displacement từ cột 'Động cơ' Bonbanh
    Ví dụ: 'Xăng 2.0 L', 'Dầu 2.4L', 'Hybrid 1.5 L'
    Trả về float: 2.0, 2.4, 1.5
    """
    if pd.isna(engine_text):
        return np.nan
    match = re.search(r"(\d+(\.\d+)?)\s*[lL]", engine_text)
    if match:
        return float(match.group(1))
    return np.nan

# Engine từ cột 'Động cơ'
bonbanh_selected['engine'] = bonbanh_selected['fuel_type'].apply(extract_engine)

# 5. Chuẩn hóa số chỗ

In [ ]:
chotot_selected['seats'] = (
    chotot_selected['seats']
    .astype(str)                          # đảm bảo là string
    .str.extract(r'(\d+)')                # lấy số đầu tiên
    .astype('Int64')                       # chuyển thành số nguyên, null giữ nguyên
)

bonbanh_selected['seats'] = (
    bonbanh_selected['seats']
    .astype(str)                          # đảm bảo là string
    .str.extract(r'(\d+)')                # lấy số đầu tiên
    .astype('Int64')                       # chuyển thành số nguyên, null giữ nguyên
)

# 6. Chuẩn hóa số năm

In [ ]:
chotot_selected['year'] = (
    chotot_selected['year']
    .astype(str)                          # đảm bảo là string
    .str.extract(r'(\d+)')                # lấy số đầu tiên
    .astype('Int64')                       # chuyển thành số nguyên, null giữ nguyên
)

bonbanh_selected['year'] = (
    bonbanh_selected['year']
    .astype(str)                          # đảm bảo là string
    .str.extract(r'(\d+)')                # lấy số đầu tiên
    .astype('Int64')                       # chuyển thành số nguyên, null giữ nguyên
)

# 7. Chuẩn hóa địa chỉ

In [ ]:
CITIES_VN = [
    "Hà Nội", "HN", "Hồ Chí Minh", "HCM", "Hải Phòng", "Đà Nẵng", "Cần Thơ",
    "An Giang", "Bà Rịa - Vũng Tàu", "Bắc Giang", "Bắc Kạn", "Bạc Liêu",
    "Bắc Ninh", "Bến Tre", "Bình Định", "Bình Dương", "Bình Phước", "Bình Thuận",
    "Cà Mau", "Cao Bằng", "Đắk Lắk", "Đắk Nông", "Điện Biên", "Đồng Nai",
    "Đồng Tháp", "Gia Lai", "Hà Giang", "Hà Nam", "Hà Tĩnh", "Hải Dương",
    "Hậu Giang", "Hòa Bình", "Hưng Yên", "Khánh Hòa", "Kiên Giang", "Kon Tum",
    "Lai Châu", "Lâm Đồng", "Lạng Sơn", "Lào Cai", "Long An", "Nam Định",
    "Nghệ An", "Ninh Bình", "Ninh Thuận", "Phú Thọ", "Quảng Bình", "Quảng Nam",
    "Quảng Ngãi", "Quảng Ninh", "Quảng Trị", "Sóc Trăng", "Sơn La", "Tây Ninh",
    "Thái Bình", "Thái Nguyên", "Thanh Hóa", "Thừa Thiên Huế", "Tiền Giang",
    "Trà Vinh", "Tuyên Quang", "Vĩnh Long", "Vĩnh Phúc", "Yên Bái"
]

def extract_city(addr):
    """
    Tách tỉnh/thành phố từ địa chỉ.
    - Nếu tìm thấy trong danh sách CITIES_VN → trả về tên chuẩn.
    - Nếu không → trả về phần sau dấu phẩy cuối cùng.
    """

    if pd.isna(addr) or str(addr).strip() == "":
        return None

    addr = str(addr)

    # 1. Tìm trong danh sách tỉnh/thành
    for city in CITIES_VN:
        if city in addr:
            if city == 'HCM':
                return 'Hồ Chí Minh'
            elif city == 'HN':
                return 'Hà Nội'
            return city

    # 2. Không tìm thấy → trả về phần sau dấu phẩy cuối
    if "," in addr:
        return addr.split(",")[-1].strip()

    # 3. Không có dấu phẩy → trả nguyên chuỗi
    return addr.strip()


chotot_selected['location'] = chotot_selected['location'].apply(extract_city)
bonbanh_selected['location'] = bonbanh_selected['location'].apply(extract_city)

# 8. Chuẩn hóa loại nhiên liệu

In [ ]:
# Chuẩn hóa fuel_type: chỉ giữ loại nhiên liệu (Xăng, Dầu, Điện, ...), bỏ dung tích
bonbanh_selected['fuel_type'] = bonbanh_selected['fuel_type'].apply(
    lambda x: str(x).split()[0] if pd.notna(x) and str(x).strip() != '' else None
)

# 9. Phân tách brand của bonbanh

In [ ]:
BRANDS = chotot_selected['brand']

# Các từ khóa hộp số (chỉ dùng để nhận diện, sau đó sẽ phân loại)
TRANSMISSIONS = ["AT", "MT", "CVT", "DAT", "ATH"]

# Các từ khóa dẫn động
DRIVES = ["4x2", "4x4", "2WD", "AWD", "HTRAC", "4Matic", "xDrive"]

# Regex động cơ
ENGINE_REGEX = r"\d(\.\d)?L?(\s?[IV]{1,3})?([A-Za-z]*)?"  # Ví dụ: 2.0L, 3.5 V6, 1.0 T, 1.8HEV

def split_series(title):
    """
    Tách chuỗi tiêu đề xe thành:
    brand | series | engine | transmission | drive | version

    - Loại bỏ từ 'Xe' ở đầu nếu có
    - Transmission trả về: Automatic hoặc Manual
    - Trích xuất brand, series, gearbox từ bonbanh_selected['title']
    """
    if pd.isna(title) or title.strip() == "":
        return pd.Series([None]*6,
                         index=["brand","series","engine","transmission","drive","version"])

    # --- 0. Bỏ từ 'Xe' ở đầu ---
    words = title.strip().split()
    if words[0].lower() == "xe":
        title = " ".join(words[1:])

    remaining = title

    # --- 1. Tìm brand ---
    brand = None
    for b in BRANDS:
        if remaining.startswith(b):
            brand = b
            remaining = remaining[len(b):].strip()
            break

    # Nếu không tìm thấy brand, lấy từ đầu tiên của remaining làm brand
    if brand is None and remaining:
        words_remain = remaining.split()
        brand = words_remain[0]
        remaining = " ".join(words_remain[1:]).strip()


    # --- 2. Tìm transmission ---
    transmission = None
    for t in TRANSMISSIONS:
        if re.search(r'\b{}\b'.format(re.escape(t)), remaining):
            transmission = "Manual" if t == "MT" else "Automatic"
            remaining = re.sub(r'\b{}\b'.format(re.escape(t)), '', remaining).strip()
            break

    # --- 3. Tìm drive ---
    drive = None
    for d in DRIVES:
        if re.search(r'\b{}\b'.format(re.escape(d)), remaining):
            drive = d
            remaining = re.sub(r'\b{}\b'.format(re.escape(d)), '', remaining).strip()
            break

    # --- 4. Tìm engine ---
    engine = None
    engine_match = re.search(ENGINE_REGEX, remaining)
    if engine_match:
        engine = engine_match.group().strip()
        remaining = remaining.replace(engine, "").strip()

    # --- 5. Còn lại: series + version ---
    parts = remaining.split()
    if len(parts) > 1:
        series_name = " ".join(parts[:-1])
        version = parts[-1]
    elif len(parts) == 1:
        series_name = parts[0]
        version = None
    else:
        series_name = None
        version = None

    return pd.Series([brand, series_name, engine, transmission, drive, version],
                     index=["brand","series","engine","transmission","drive","version"])

parsed = bonbanh_selected['title'].apply(split_series)

bonbanh_selected['brand'] = parsed['brand']
bonbanh_selected['series'] = parsed['series']
bonbanh_selected['gear'] = parsed['transmission']

bonbanh_selected = bonbanh_selected[normalized_columns]

# 10. Lưu dữ liệu đã xử lý thành 1 file

In [ ]:
# Tạo folder nếu chưa có
os.makedirs('../data/interim', exist_ok=True)

# Gộp 2 DataFrame
normalize_df = pd.concat([chotot_selected, bonbanh_selected], ignore_index=True)

# Lưu CSV
normalize_df.to_csv('../data/interim/normalize_interim.csv', index=False, encoding='utf-8-sig')

del chotot_selected
del bonbanh_selected

print("✓ Đã lưu file normalize_interim.csv")
print("\n" + "="*50)
print("TỔNG QUAN DỮ LIỆU")
print("="*50)
print(f"\nTổng số bản ghi: {normalize_df.shape[0]}")
print(f"\nCác cột chuẩn hóa: {list(normalize_df.columns)}")
